In [1]:
import os
from pathlib import Path

# Set working directory to project root if running from 'notebooks/'
notebook_dir = Path().resolve()
project_root = notebook_dir.parent  # Assumes notebook is inside 'notebooks/'

os.chdir(project_root)
print(f"✅ Working directory set to: {project_root}")

✅ Working directory set to: F:\JCMDataCenter\Proyectos\Football_analysis


In [2]:
import pandas as pd

# === File paths
input_path = Path("data/raw/matchlogs_fbref.csv")
output_path = Path("data/processed/cleaned_matchlogs.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# === Step 1: Load raw CSV
df = pd.read_csv(input_path, dtype=str, encoding="utf-8").fillna("")

# === Step 2: Normalize player names
df["player_name"] = df["player_name"].str.replace("_", " ", regex=False).str.title()

# === Step 3: Rename columns
rename_dict = {
    "player_name": "Player_name",
    "player_id": "Player_ID",
    "season": "Seasons",
    "Date": "Date",
    "Day": "Day",
    "Comp": "Competition",
    "Round": "Round",
    "Venue": "Home_Away",
    "Result": "Result",
    "Squad": "Player_team",
    "Opponent": "Rival_team",
    "Start": "Start",
    "Pos": "Position",
    "Min": "Minutes",
    "Gls": "Goals",
    "Ast": "Assists",
    "PK": "Penalty_kick",
    "PKatt": "Penalty_kick_att",
    "Sh": "Shots",
    "SoT": "Shots_on_target",
    "CrdY": "Yellow_cards",
    "CrdR": "Red_cards",
    "Touches": "Touches",
    "Tkl": "Tackles",
    "Int": "Interceptions",
    "Blocks": "Blocks",
    "xG": "xG",
    "npxG": "non_penalty_xG",
    "xAG": "x_assisted_G",
    "SCA": "Shot_creating_actions",
    "GCA": "Goal_creating_actions",
    "Cmp": "Passes_completed",
    "Att": "Passes_att",
    "Cmp%": "Percent_passes",
    "PrgP": "Progressive_passes",
    "Carries": "Feet_control",
    "PrgC": "Progressive_control",
    "Att_2": "Dribbles_attempted",
    "Succ": "Dribbles_completed"
}

df.rename(columns={k: v for k, v in rename_dict.items() if k in df.columns}, inplace=True)

# === Step 4: Drop raw ID columns no longer needed
for col in ["Match Report", "season", "player_name", "player_id"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# === Step 5: Filter valid rows
df = df[df["Player_name"].notna() & df["Player_ID"].notna() & df["Seasons"].notna()]
if "Date" in df.columns:
    df = df[df["Date"].notna()]

# === Step 6: Drop rows where player did not play
if "Position" in df.columns:
    df = df[df["Position"] != "On matchday squad, but did not play"]

# === Step 6.5: Drop empty rows (except core identifiers)
non_core = [col for col in df.columns if col not in ["Player_name", "Player_ID", "Seasons"]]
df = df[df[non_core].apply(lambda row: any(cell.strip() for cell in row), axis=1)]

# === Step 7: Drop duplicates (same player and date)
if "Date" in df.columns:
    df = df.drop_duplicates(subset=["Player_ID", "Date"])

# === Step 8: Clean team names (remove language codes)
for col in ["Player_team", "Rival_team"]:
    if col in df.columns:
        df[col] = df[col].str.replace(r"^[a-z]{2,3}\s+", "", regex=True)

# === Step 9: Reorder columns
core = ["Player_name", "Player_ID", "Seasons"]
rest = [c for c in df.columns if c not in core]
df = df[core + rest]

# === Step 10: Save
df.to_csv(output_path, index=False, encoding="utf-8")
print(f"✅ Cleaned matchlogs saved at: {output_path} | Rows: {len(df)}")

✅ Cleaned matchlogs saved at: data\processed\cleaned_matchlogs.csv | Rows: 471
